In [33]:

'''
Generate time aligned dataset
'''
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import argparse
import pathlib
import logging
import colorlog
import os
import ipynbname
import pandas as pd

class ConsoleLogger:
    _logger = None
    
    @classmethod
    def get_logger(cls):
        if cls._logger is None:
            cls._logger = logging.getLogger(__name__)
            cls._logger.setLevel(logging.DEBUG)
            
            formatter = colorlog.ColoredFormatter(
                '[%(asctime)s] %(log_color)s%(levelname)-8s%(reset)s %(blue)s%(message)s',
                log_colors={
                    'DEBUG':    'cyan',
                    'INFO':     'green',
                    'WARNING':  'yellow',
                    'ERROR':    'red',
                    'CRITICAL': 'bold_red,bg_white',
                }
            )
            
            ch = logging.StreamHandler()
            ch.setLevel(logging.DEBUG)
            ch.setFormatter(formatter)
            
            cls._logger.addHandler(ch)
        return cls._logger
console = ConsoleLogger.get_logger()

# temporary input
target_workspace = pathlib.Path(os.getcwd()) / "2024-12-13-11-38-56"
target_nback_response = pathlib.Path(os.getcwd()) / "nback_response_aligned.csv"

timestamp_base = "timestamp_0.csv"
start_unixtime_timestamp = 0.0
    
def change_relative_video_timestamp(input_csv: str, output: str) -> bool:
    try:
        df_video_timestamp = pd.read_csv(input_csv, header=None)
        df_rebase_video_timestamp = pd.DataFrame()

        if df_video_timestamp.empty:
            console.error("CSV file is empty")
            return False

        global start_unixtime_timestamp
        start_unixtime_timestamp = float(df_video_timestamp.iloc[0, 0])
        df_rebase_video_timestamp[0] = df_video_timestamp[0] - start_unixtime_timestamp
        df_rebase_video_timestamp[0] = df_rebase_video_timestamp[0].map(lambda x: f"{x:.6f}")
        df_rebase_video_timestamp.to_csv(output, header=False, index=False)
        return True

    except Exception as e:
        console.error("Error : {e}")
        return False


def change_nback_response_timestamp(input_csv:str, output:str, start_unixtime:float, offset:float) -> bool:
    df_nback_timestamp = pd.read_csv(input_csv, header=None)
    df_nback_timestamp[0] = df_nback_timestamp[0] - start_unixtime + offset
    df_nback_timestamp.columns = ['time', 'event']
    df_nback_timestamp.to_csv(output, index=False, header=True)
    return True

# perform
change_relative_video_timestamp(target_workspace / 'camera/timestamp_0.csv', target_workspace / 'camera/timestamp_0_rel.csv')
change_nback_response_timestamp(target_workspace / 'nback_response.csv', target_workspace / 'nback_response_aligned.csv', start_unixtime_timestamp, -0.6716)

True

In [14]:
import pandas as pd

def find_csv_file(start_dir, filename:str):
    for root, dirs, files in os.walk(start_dir):
        if filename in files:
            return os.path.abspath(os.path.join(root, filename))
    return None

# required files
required_files = ["timestamp_0.csv", "timestamp_2.csv", "timestamp_5.csv", "timestamp_6.csv", "scenario_history.csv", "nback_response.csv", "blinks.csv", "3d_eye_states.csv", "fixations.csv", "gaze.csv", "saccades.csv", "world_timestamps.csv", "events.csv"]
required_file_path = {}
# find all required files in input path
for file in required_files:
    file_path = find_csv_file(input_path, file)
    if file_path is None:
        console.error(f"Required file '{file}' not found in '{input_path}'.")
    required_file_path[file] = file_path

    #first_column = df.iloc[:, 0]


# start and end time
start_timestamp = pd.read_csv(required_file_path["scenario_history.csv"], header=None).iloc[0,0]
end_timestamp = pd.read_csv(required_file_path["scenario_history.csv"], header=None).iloc[-1,0]

# manipulate nback response
def convert_unix_to_relative(input_csv, output_csv):
    with open(input_csv, newline='', encoding='utf-8') as f:
        reader = list(csv.reader(f))

    if not reader:
        print("CSV 파일이 비어 있습니다.")
        return

    base_time = float(reader[0][0])
    converted_rows = []

    for row in reader:
        try:
            unix_time = float(row[0])
            relative_time = unix_time - base_time
            converted_rows.append([f"{relative_time:.3f}"] + row[1:])
        except (IndexError, ValueError):
            print(f"행 무시됨: {row}")

    with open(output_csv, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(converted_rows)

    print(f"상대 시간으로 변환된 CSV가 저장되었습니다: {output_csv}")


# nback response alignment
df = pd.read_csv(required_file_path["nback_response.csv"], header=None)
df[0] = df[0] - start_timestamp
df.columns = ['time', 'event']
directory = os.path.dirname(required_file_path["nback_response.csv"])
save_path = os.path.join(directory, "nback_response_aligned.csv")
df.to_csv(save_path, index=False, header=True)
